#### Pintle test Omega 600 PSI calibration test

This code was create to find a linear regression for Omega 600 PSI calibration data in order to create a well calibrated pressure transducer.

Author: Mark Musil B.S. E.E. Portland State Univ. March 2018

In [26]:
#imports
import numpy as np
psi = [0,10,20,30,40,50,60,70,80,90]
analogout = [171,180,190,196,205,
             214, 222, 232, 240, 248]
diffarray = []
for i in range(len(psi)):
    if i == 0:
        bert = 'weird'
    else:
        diffarray.append((analogout[i]-analogout[i-1])/10)
print ('The array of differences is: ' + str(diffarray))
print('The conversion of arduino output to'
      ' psi is: ' + str(np.average(diffarray)))


The array of differences is: [0.9, 1.0, 0.6, 0.9, 0.9, 0.8, 1.0, 0.8, 0.8]
The conversion of arduino output to psi is: 0.855555555556
